# BEV simulation preparation
Check a variety of MATSim output data of trajectories and prepare BEV simulation.

In [ ]:
import tqdm
%load_ext autoreload
%autoreload 2
%cd D:\synthetic-sweden
%matplotlib inline

In [ ]:
# Load libs
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import matsim
from collections import defaultdict
import lib.dataworkers as workers

In [ ]:
scenario = 'scenario_vg_car'
file_path2output = f'dbs/{scenario}/output/'

In [ ]:
# read network
net = matsim.read_network(file_path2output + 'output_network.xml.gz')
geo = net.as_geo()  # combines links+nodes into a Geopandas dataframe with LINESTRINGs

## 1. Traffic on the road network

In [ ]:
events = matsim.event_reader(file_path2output + 'output_events.xml.gz', types='entered link,left link')

link_counts = defaultdict(int) # defaultdict creates a blank dict entry on first reference

for event in tqdm(events, desc='Streaming events'):
    if event['type'] == 'entered link':
        link_counts[event['link']] += 1

# convert our link_counts dict to a pandas dataframe,
# with 'link_id' column as the index and 'count' column with value:
link_counts = pd.DataFrame.from_dict(link_counts, orient='index', columns=['count']).rename_axis('link_id')

# attach counts to our Geopandas network from above
volumes = geo.merge(link_counts, on='link_id')
volumes.plot(column='count', figsize=(10,10), cmap='Wistia') #cmap is colormap

In [ ]:
volumes.to_file(f'dbs/output_summary/{scenario}/volumes.shp')

## 2. Parking opportunities

In [ ]:
# Load output plan
plans = matsim.plan_reader(file_path2output + 'output_experienced_plans.xml.gz', selectedPlansOnly=True)
# Aggregate all individuals' plans
df_output = workers.plans_summary(pd.concat([workers.personplan2df(person, plan, experienced=True) for person, plan in plans]))

In [ ]:
df_output_car = df_output.loc[df_output['mode'] == 'car', ['PId', 'act_id', 'act_purpose', 'dep_time', 'trav_time_min', 'act_time', 'act_end', 'POINT_X', 'POINT_Y']]

In [ ]:
df_output_car.loc[:, 'home'] = df_output_car.loc[:, 'act_purpose'].apply(lambda x: 1 if x == 'home' else 0)
df_parking = df_output_car.groupby(['POINT_X', 'POINT_Y', 'home'])['act_time'].sum().reset_index(name ='parking_time_min')

In [ ]:
df_parking.to_csv(f'dbs/output_summary/{scenario}/parking_time.csv', index=False)

## 3. Get individual trajectories for BEV simulation

| Event type             | Information                                          | Note                   |
|------------------------|------------------------------------------------------|------------------------|
| actend                 | person, link, actType                                | 0                      |
| departure              | person, link, legMode                                | 0                      |
| PersonEntersVehicle    | person, vehicle                                      | 0, person=vehicle      |
| vehicle enters traffic | person, link, vehicle, networkMode, relativePosition | 0                      |
| left link              | link, vehicle                                        | 1                      |
| entered link           | link, vehicle                                        | 1                      |
| travelled              | person, distance, mode                               | 1, mode=pt, bike, walk |
| arrival                | person, link, legMode                                | 2                      |
| actstart               | person, link, x, y, actType                          | 2                      |
| vehicle leaves traffic | person, link, vehicle, networkMode, relativePosition | 2                      |
| PersonLeavesVehicle    | person, vehicle                                      | 2                      |

Stage 0: 'actend', 'departure', 'PersonEntersVehicle', 'vehicle enters traffic'

Stage 1: 'left link', 'entered link', 'travelled'

Stage 2: 'arrival', 'actstart', 'vehicle leaves traffic', 'PersonLeavesVehicle'


In [ ]:
# Create event batches for BEV simulation
workers.matsim_events2database(scenario=scenario)
workers.eventsdb2batches(scenario=scenario, batch_num=20)

## 4 Get slope for the road network
The volumes network is processed with DEM data using `add_slope.model3` in QGIS to get slope information.

1) Split lines by maximum distance = 500 m.
2) Reproject to 3035.
3) Get elevation information (Drape).
4) Extract Z value.
5) Calculate slope by (z_last - z_first) / length_seg
6) Reproject to 3006.
5) Add line lengths.

In [ ]:
network = gpd.read_file(f'dbs/output_summary/{scenario}/volumes_slope.shp')
network.head()

#### 4.1.1 Set length_seg < 30 m roads' slope to 0

In [ ]:
network.loc[network['length_seg'] < 30, 'slope'] = 0
len(network.loc[network['slope'] < -0.06, :]) / len(network), len(network.loc[network['slope'] > 0.06, :]) / len(network)

#### 4.1.2 Set roads' slope not in \[-6%, 6%] to 6% / -6%

In [ ]:
network.loc[network['slope'] < -0.06, 'slope'] = -0.06
network.loc[network['slope'] > 0.06, 'slope'] = 0.06
network.loc[:, 'slope'] *= 100

## 5. Access to home charger

In [ ]:
activity_purpose_dict = {1: 'h', 4: 'w', 10: 'o', 6: 's'}
df_plan = pd.read_pickle('dbs/agents/df_act_plan_vgr.pkl')
df_plan = df_plan.loc[:, ['PId', 'act_purpose', 'building_type']]
df_plan['act_purpose'] = df_plan['act_purpose'].map(activity_purpose_dict)
df_plan = df_plan.loc[df_plan['act_purpose'] == 'h', :].drop_duplicates(subset=['PId', 'act_purpose'])

# Detached houses coding
building_dict = {130: 1, 131: 1, 132: 1}
df_plan.loc[:, 'home_charger'] = df_plan.loc[:, 'building_type'].apply(lambda x: building_dict[x] if x in building_dict else 0)
print("Share of access to home charger: %.2f %%"%(len(df_plan.loc[df_plan['home_charger'] == 1, :]) / (len(df_plan)) * 100))

# Save home charger information
df_plan.loc[:, ['PId', 'building_type', 'home_charger']].to_csv(f'dbs/output_summary/home_charger_access.csv', index=False)

## 6. Charging opportunities

In [ ]:
file_path2output_summary = f'dbs/output_summary/{scenario}/'
# Focus only on valid agents' trips
agents = pd.read_csv(file_path2output_summary + "valid_agents.csv")
df_trips = pd.read_csv(file_path2output_summary + 'trips.csv')
df_trips_input = df_trips.loc[(df_trips['src'] == 'input') & (df_trips['PId']).isin(agents.PId), ['PId', 'act_id', 'Deso']]
df_trips = df_trips.loc[(df_trips['src'] == 'output') & (df_trips['PId']).isin(agents.PId), :]

In [ ]:
# Fill start location
def fill_start_location(data):

    x = data.loc[:, 'POINT_X'].values
    x[0] = x[-1]
    y = data.loc[:, 'POINT_Y'].values
    y[0] = y[-1]
    data.loc[:, 'POINT_X'] = x
    data.loc[:, 'POINT_Y'] = y
    return data
tqdm.pandas()
df_trips = df_trips.groupby('PId').progress_apply(fill_start_location).reset_index(drop=True)

Add act_start and deso zones from input, and save data.

In [ ]:
df_trips.loc[:, 'act_start'] = round((df_trips.loc[:, 'dep_time'] + df_trips.loc[:, 'trav_time_min'] / 60) * 60)
df_trips.loc[:, 'act_start'] = df_trips.loc[:, 'act_start'].astype(int)
df_trips_pro = df_trips.drop(columns=['Deso']).merge(df_trips_input, how='left', on=['PId','act_id']).rename(columns={'Deso': 'deso'})
df_trips_pro.to_csv(file_path2output_summary + 'charging_opportunity.csv', index=False)